In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from tqdm import tqdm

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd 'drive/My Drive/GenomicsProject/GenomicsProject'

/content/drive/My Drive/GenomicsProject/GenomicsProject


In [ ]:
import numpy as np

# Keys to npzfile of train & eval
train_cells = ['E065', 'E004', 'E066', 'E005', 'E012', 'E027', 'E053', 'E013', 'E028', 'E061', 'E109', 'E120', 'E062', 'E037', 'E038', 'E024', 'E105', 'E011', 'E106', 'E082', 'E097', 'E116', 'E098', 'E058', 
'E117', 'E059', 'E070', 'E118', 'E085', 'E104', 'E119', 'E006', 'E127', 'E047', 'E094', 'E007', 'E054', 'E128', 'E095', 'E055', 'E114', 'E100', 'E056', 'E016', 'E122', 'E057', 'E123', 'E079', 'E003', 'E050']

eval_cells = ['E065', 'E004', 'E066', 'E005', 'E012', 'E027', 'E053', 'E013', 'E028', 'E061', 'E109', 'E120', 'E062', 'E037', 'E038', 'E024', 'E071', 'E105', 'E087', 'E011', 'E106', 'E096', 'E082', 'E097', 
'E116', 'E098', 'E058', 'E117', 'E084', 'E059', 'E070', 'E118', 'E085', 'E104', 'E119', 'E006', 'E112', 'E127', 'E047', 'E094', 'E007', 'E054', 'E113', 'E128', 'E095', 'E055', 'E114', 'E100', 'E056', 'E016', 'E122', 'E057', 'E123', 'E079', 'E003', 'E050']

# Load data
train_data = np.load('train.npz')
eval_data = np.load('eval.npz')

# Combine Train Data to use information from all cells
train_inputs = [] # Input histone mark data
train_outputs = [] # Correct expression value
for cell in train_cells:
    cell_data = train_data[cell]
    hm_data = cell_data[:,:,1:6]
    exp_values = cell_data[:,0,6]
    train_inputs.append(hm_data)
    train_outputs.append(exp_values)

train_inputs = np.concatenate(train_inputs, axis=0)
train_outputs = np.concatenate(train_outputs, axis=0)

# Prepare Eval inputs in similar way
eval_inputs = []
for cell in eval_cells:
    cell_data = eval_data[cell]
    hm_data = cell_data[:,:,1:6]
    eval_inputs.append(hm_data)

eval_inputs = np.concatenate(eval_inputs, axis=0)


# Example submission of predicted values
import pandas as pd

cell_list = []
gene_list = []
example_eval_preds = np.random.randn(len(eval_inputs))
for cell in eval_cells:
    cell_data = eval_data[cell]
    cell_list.extend([cell]*len(cell_data))
    genes = cell_data[:,0,0].astype('int32')
    gene_list.extend(genes)

id_column = [] # ID is {cell type}_{gene id}
for idx in range(len(eval_inputs)):
    id_column.append(f'{cell_list[idx]}_{gene_list[idx]}')

df_data = {'id': id_column, 'expression' : example_eval_preds}
submit_df = pd.DataFrame(data=df_data)

submit_df.to_csv('sample_submission.csv', header=True, index=False, index_label=False)

train_inputs_flat = train_inputs.reshape((800000,500))

In [ ]:
train_inputs.shape

(800000, 100, 5)

In [ ]:
train_outputs.shape

(800000,)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train_inputs_flat[:-1000,:],train_outputs[:-1000])
print(reg.score(train_inputs_flat[-1000:,:],train_outputs[-1000:]))
ypred = reg.predict(train_inputs_flat[-1000:,:])
np.sum((ypred - train_outputs[-1000:])*(ypred - train_outputs[-1000:]))/1000

-0.011505315544347418


2.83518212890625

In [ ]:
# Prepare Eval inputs in similar way
eval_inputs = []
for cell in eval_cells:
    cell_data = eval_data[cell]
    hm_data = cell_data[:,:,1:6]
    eval_inputs.append(hm_data)

eval_inputs = np.concatenate(eval_inputs, axis=0)

eval_inputs_flat = eval_inputs.reshape(-1,500)


# Example submission of predicted values
import pandas as pd

cell_list = []
gene_list = []
#example_eval_preds = np.random.randn(len(eval_inputs))
eval_preds = reg.predict(eval_inputs_flat)
for cell in eval_cells:
    cell_data = eval_data[cell]
    cell_list.extend([cell]*len(cell_data))
    genes = cell_data[:,0,0].astype('int32')
    gene_list.extend(genes)

id_column = [] # ID is {cell type}_{gene id}
for idx in range(len(eval_inputs)):
    id_column.append(f'{cell_list[idx]}_{gene_list[idx]}')

df_data = {'id': id_column, 'expression' : example_eval_preds}
submit_df = pd.DataFrame(data=df_data)

submit_df.to_csv('linreg_submission.csv', header=True, index=False, index_label=False)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor, DistanceMetric
knn = KNeighborsRegressor(n_neighbors = 5, weights = 'uniform')
knn.fit(train_inputs_flat[:-1000,:], train_outputs[:-1000])
knn_accuracy = knn.score(train_inputs_flat[-1000:], train_outputs[-1000:])
print("The R^2 value is ",knn_accuracy)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 70, max_depth = 15, n_jobs=-1)
rf.fit(train_inputs_flat[:750000,:],train_outputs[:750000])
rf_accuracy = rf.score(train_inputs_flat[750000:,:],train_outputs[750000:])
print('The R^2 value is ',rf_accuracy)

The R^2 value is  0.04075189112004518


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 70, max_depth = 15, n_jobs=-1)
rf.fit(train_inputs_flat[:750000,:],train_outputs[:750000])
rf_accuracy = rf.score(train_inputs_flat[750000:,:],train_outputs[750000:])
print('The R^2 value is ',rf_accuracy)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 200, max_depth = 15, n_jobs=-1)
rf.fit(train_inputs_flat[:750000,:],train_outputs[:750000])
rf_accuracy = rf.score(train_inputs_flat[750000:,:],train_outputs[750000:])
print('The R^2 value is ',rf_accuracy)

# Prepare Eval inputs in similar way
eval_inputs = []
for cell in eval_cells:
    cell_data = eval_data[cell]
    hm_data = cell_data[:,:,1:6]
    eval_inputs.append(hm_data)

eval_inputs = np.concatenate(eval_inputs, axis=0)

eval_inputs_flat = eval_inputs.reshape(-1,500)


# Example submission of predicted values
import pandas as pd

cell_list = []
gene_list = []
#example_eval_preds = np.random.randn(len(eval_inputs))
eval_preds = reg.predict(eval_inputs_flat)
for cell in eval_cells:
    cell_data = eval_data[cell]
    cell_list.extend([cell]*len(cell_data))
    genes = cell_data[:,0,0].astype('int32')
    gene_list.extend(genes)

id_column = [] # ID is {cell type}_{gene id}
for idx in range(len(eval_inputs)):
    id_column.append(f'{cell_list[idx]}_{gene_list[idx]}')

df_data = {'id': id_column, 'expression' : example_eval_preds}
submit_df = pd.DataFrame(data=df_data)

submit_df.to_csv('forrest_submission2.csv', header=True, index=False, index_label=False)

The R^2 value is  0.06895683218168613
